In [1]:
import pandas as pd
import plotly.express as px
import re
import datetime
import numpy as np
import json
from Project.Database import Db

In [2]:
level1_json = json.load(
    open(Db.get_project_path("Project/04TPMAlgorithm/TPM/output/Experiment_minsup0.14_minconf_0.5/level1.json")))

In [3]:
level2_json = json.load(
    open(Db.get_project_path("Project/04TPMAlgorithm/TPM/output/Experiment_minsup0.14_minconf_0.5/level2.json")))

In [4]:
level3_json = json.load(
    open(Db.get_project_path("Project/04TPMAlgorithm/TPM/output/Experiment_minsup0.14_minconf_0.5/level3.json")))

In [5]:
meta = Db.load_data(meta=True, consumption=False, hourly=False)

In [6]:
def json_to_dataframe(json_file):
    if "," in json_file[0]["name_node"]:
        level_df = pd.DataFrame(columns=["pattern","supp","conf", "time"])
        for i in json_file:
            for j in i["patterns"]:
                level_df.loc[level_df.shape[0]] = j
    else:
        level_df = pd.DataFrame(columns=["name_node","supp","conf"])
        for i in json_file:
            level_df.loc[level_df.shape[0]] = i
    return level_df

In [7]:
level2_df = json_to_dataframe(level2_json)

In [8]:
level3_df = json_to_dataframe(level3_json)
level3_df

,pattern,supp,conf,time
0,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.698630,0.704420,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
1,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
2,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.701370,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
3,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
4,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
...,...,...,...,...
5086,Load_StatusPlugLoadHeatingPad>Load_StatusMBALi...,0.701370,0.707182,"{'337': [[['2014-06-03 21:57:29', '2014-06-03 ..."
5087,Load_StatusPlugLoadHeatingPad>Load_StatusMBALi...,0.701370,0.701370,"{'337': [[['2014-06-03 21:57:29', '2014-06-03 ..."
5088,Load_StatusPlugLoadHeatingPad->Load_StatusSens...,0.698630,0.704420,"{'337': [[['2014-06-03 21:57:29', '2014-06-03 ..."
5089,Load_StatusApplianceOven>Load_StatusPlugLoadVi...,0.276712,0.651613,"{'13': [[['2013-07-14 08:14:55', '2013-07-14 1..."


In [9]:
level3_df_follow_restricted = level3_df.loc[level3_df['pattern'].str.findall('-').map(len) <= 1]

In [38]:
re.findall('[\w_]+', level3_df_follow_restricted['pattern'].iloc[0])

['Load_StatusMBALights',
 'Load_StatusSensHeatPrntBDOWN',
 'Load_StatusSensHeatPrntBDOWN',
 'Load_StatusPlugLoadBlender',
 'Load_StatusMBALights',
 'Load_StatusPlugLoadBlender']

In [59]:
index_list = []
for index, row in level3_df_follow_restricted.iterrows():
    tmp_floor_set = set()
    appliance_check_list = list()
    for col in set(re.findall('[\w_]+', row['pattern'])):
        try:
            tmp_floor_set.add(meta.loc[col, 'Measurement_Floor'])
            appliance_check_list.append('SensHeat' not in col)
        except:
            continue
    if '1stFloor' in tmp_floor_set and '2ndFloor' in tmp_floor_set and sum(appliance_check_list) > 1:
        attribute_list = set(re.findall('[\w_]+', row['pattern']))
        if sum(appliance_check_list) == 3:
            continue
        else:
            index_list.append(index)
print(len(index_list))

714


In [60]:
level3_df_follow_restricted.loc[index_list]

,pattern,supp,conf,time
6,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
7,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
8,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.698630,0.704420,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
9,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
12,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'262': [[['2014-03-20 05:58:46', '2014-03-20 ..."
...,...,...,...,...
4950,Load_StatusSensHeatChildBUP|Load_StatusPlugLoa...,0.547945,0.552486,"{'337': [[['2014-06-03 00:03:30', '2014-06-03 ..."
4951,Load_StatusSensHeatChildBUP|Load_StatusPlugLoa...,0.827397,0.834254,"{'337': [[['2014-06-03 00:03:30', '2014-06-03 ..."
5051,Load_StatusSensHeatChildBUP>Load_StatusPlugLoa...,0.846575,0.853591,"{'73': [[['2013-09-12 18:29:35', '2013-09-12 1..."
5053,Load_StatusSensHeatChildBUP>Load_StatusPlugLoa...,0.989041,1.000000,"{'73': [[['2013-09-12 18:29:35', '2013-09-12 1..."


In [65]:
def selected_appliances(wanted_attributes: list, only_appliance=False, include_residents=True) -> tuple:
    person_appliance_rule_indexes = []
    appliance_appliance_rule_indexes = []
    for index, row in level3_df_follow_restricted.iterrows():
        tmp_floor_set = set()
        appliance_check_list = list()
        person_check_list = list()
        for col in set(re.findall('[\w_]+', row['pattern'])):
            try:
                tmp_floor_set.add(meta.loc[col, 'Measurement_Floor'])
                person_check_list.append('SensHeat' in col)
                appliance_check_list.append('SensHeat' not in col)
            except:
                continue
        if '1stFloor' in tmp_floor_set and '2ndFloor' in tmp_floor_set and sum(appliance_check_list) >= 1:
            attribute_list = set(re.findall('[\w_]+', row['pattern']))
            if sum(person_check_list) >= 1:
                person_appliance_rule_indexes.append(index)
            else:
                appliance_appliance_rule_indexes.append(index)
    print(len(appliance_appliance_rule_indexes))

350


In [ ]:
for i in level3_df_follow_restricted.loc[appliance_appliance_rule_indexes]:

In [ ]:
from _04TPMA

In [11]:
meta.loc[meta.index == "Load_StatusPlugLoadBlender"]

,Subsystem,Measurement_Location,Parameter,Description,Units,Aggregation_Method,min_value,max_value,Measurement_Floor,Load_Match,Status_Match,Elec_Match
Unnamed: 0,,,,,,,,,,,,
Load_StatusPlugLoadBlender,Loads,Kitchen,Status_OnOff,Number to indicate whether blender is activate...,BinaryStatus,Average,-1.11,1.0,1stFloor,Load_KPlugLoadsPowerUsage,NaN,Elec_PowerPlugsKitRange
